<a href="https://colab.research.google.com/github/MarcoBrac/Test/blob/master/test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!head TABLA_MODELO_HIT_3.csv

CONTRATO,NUM_CLIENTE,BC_SCORE,COSECHA,EDAD,OCPUACION_SEXO,CUENTA_MAS_ANTIGUA_BANCO,MAX_MOP_6M_AB_TDC,CUENTA_MAS_ANTIGUA_TDC,SALDO_VENCIDO_ABIERTAS,MAX_MOP_6M,PORC_UTILIZ_ENTRE_CUENT_REV_AB,PRODUCTO,SUB_PRODUCTO,VINTAGE,AMOUNT,FLAG_1PV_3,FLAG_2PV_6,FLAG_3PV_12,FLAG_3PV_18,FLAG_DESEMBOLSADO
400015808,1953462,660,201505,37,C_CON NEGOCIO/F,31,NULL,NULL,0,9,NULL,OMEGA,OMEGA_M,201505,11728,0,1,1,1,1
400015823,772865,663,201505,64,F_JUBILADO/F,94,2,187,511,2,0.219478426,OMEGA,OMEGA_M,201505,17191,0,0,0,0,1
400015912,3873005,561,201505,41,A_CON NEGOCIO/M,28,NULL,NULL,4662,7,NULL,OMEGA,OMEGA_M,201505,5897,0,0,0,0,1
400015973,776366,724,201505,44,C_CON NEGOCIO/F,NULL,1,18,0,1,0.104229355,OMEGA,OMEGA_M,201505,8000,0,0,0,0,1
400014820,347497,713,201505,40,C_CON NEGOCIO/F,49,1,42,0,1,0.034421836,OMEGA,OMEGA_M,201505,29686,0,0,0,0,1
400018477,1011052,631,201506,53,B_EMPLEADO O ASALARIADO/M,NULL,1,137,48701,9,0,OMEGA,OMEGA_M,201506,13615,0,0,0,0,1
400018510,1800455250,661,201506,50,C_CON NEGOCIO/F,11

In [0]:
from google.colab import files


In [0]:
uploaded = files.upload()

Saving TABLA_MODELO_HIT_2.xlsx to TABLA_MODELO_HIT_2.xlsx


In [0]:
import pandas as pd
import numpy as np


In [0]:
df1 = pd.read_csv('TABLA_MODELO_HIT_3.csv' ) 

In [0]:
df1.head()

,CONTRATO,NUM_CLIENTE,BC_SCORE,COSECHA,EDAD,OCPUACION_SEXO,CUENTA_MAS_ANTIGUA_BANCO,MAX_MOP_6M_AB_TDC,CUENTA_MAS_ANTIGUA_TDC,SALDO_VENCIDO_ABIERTAS,MAX_MOP_6M,PORC_UTILIZ_ENTRE_CUENT_REV_AB,PRODUCTO,SUB_PRODUCTO,VINTAGE,AMOUNT,FLAG_1PV_3,FLAG_2PV_6,FLAG_3PV_12,FLAG_3PV_18,FLAG_DESEMBOLSADO
0,400015808,1953462,660,201505,37.0,C_CON NEGOCIO/F,31.0,NaN,NaN,0.0,9.0,NaN,OMEGA,OMEGA_M,201505,11728.0,0,1,1,1,1
1,400015823,772865,663,201505,64.0,F_JUBILADO/F,94.0,2.0,187.0,511.0,2.0,0.219478,OMEGA,OMEGA_M,201505,17191.0,0,0,0,0,1
2,400015912,3873005,561,201505,41.0,A_CON NEGOCIO/M,28.0,NaN,NaN,4662.0,7.0,NaN,OMEGA,OMEGA_M,201505,5897.0,0,0,0,0,1
3,400015973,776366,724,201505,44.0,C_CON NEGOCIO/F,NaN,1.0,18.0,0.0,1.0,0.104229,OMEGA,OMEGA_M,201505,8000.0,0,0,0,0,1
4,400014820,347497,713,201505,40.0,C_CON NEGOCIO/F,49.0,1.0,42.0,0.0,1.0,0.034422,OMEGA,OMEGA_M,201505,29686.0,0,0,0,0,1


In [0]:
df1['CONTRATO']

df2 = df1.copy()

In [0]:
df2.head()

,CONTRATO,NUM_CLIENTE,BC_SCORE,COSECHA,EDAD,OCPUACION_SEXO,CUENTA_MAS_ANTIGUA_BANCO,MAX_MOP_6M_AB_TDC,CUENTA_MAS_ANTIGUA_TDC,SALDO_VENCIDO_ABIERTAS,MAX_MOP_6M,PORC_UTILIZ_ENTRE_CUENT_REV_AB,PRODUCTO,SUB_PRODUCTO,VINTAGE,AMOUNT,FLAG_1PV_3,FLAG_2PV_6,FLAG_3PV_12,FLAG_3PV_18,FLAG_DESEMBOLSADO,column1_
0,400015808,1953462,660,201505,37.0,C_CON NEGOCIO/F,31.0,NaN,NaN,0.0,9.0,NaN,OMEGA,OMEGA_M,201505,11728.0,0,1,1,1,1,6
1,400015823,772865,663,201505,64.0,F_JUBILADO/F,94.0,2.0,187.0,511.0,2.0,0.219478,OMEGA,OMEGA_M,201505,17191.0,0,0,0,0,1,6
2,400015912,3873005,561,201505,41.0,A_CON NEGOCIO/M,28.0,NaN,NaN,4662.0,7.0,NaN,OMEGA,OMEGA_M,201505,5897.0,0,0,0,0,1,6
3,400015973,776366,724,201505,44.0,C_CON NEGOCIO/F,NaN,1.0,18.0,0.0,1.0,0.104229,OMEGA,OMEGA_M,201505,8000.0,0,0,0,0,1,6
4,400014820,347497,713,201505,40.0,C_CON NEGOCIO/F,49.0,1.0,42.0,0.0,1.0,0.034422,OMEGA,OMEGA_M,201505,29686.0,0,0,0,0,1,6


In [0]:
def bins_1(row):
    if row.EDAD <= 40:
        return '<=40'
    elif row.EDAD <= 50:
        return '<=50'
    else:
        return '> 50'

In [0]:
df2["v5"] = df2.apply(bins_1, axis=1)
df2['edad2'] = df2.EDAD

In [0]:
df2.head(20)

,CONTRATO,NUM_CLIENTE,BC_SCORE,COSECHA,EDAD,OCPUACION_SEXO,CUENTA_MAS_ANTIGUA_BANCO,MAX_MOP_6M_AB_TDC,CUENTA_MAS_ANTIGUA_TDC,SALDO_VENCIDO_ABIERTAS,MAX_MOP_6M,PORC_UTILIZ_ENTRE_CUENT_REV_AB,PRODUCTO,SUB_PRODUCTO,VINTAGE,AMOUNT,FLAG_1PV_3,FLAG_2PV_6,FLAG_3PV_12,FLAG_3PV_18,FLAG_DESEMBOLSADO,column1_,v5,edad2
0,400015808,1953462,660,201505,37.0,C_CON NEGOCIO/F,31.0,NaN,NaN,0.0,9.0,NaN,OMEGA,OMEGA_M,201505,11728.0,0,1,1,1,1,6,<=40,37.0
1,400015823,772865,663,201505,64.0,F_JUBILADO/F,94.0,2.0,187.0,511.0,2.0,0.219478,OMEGA,OMEGA_M,201505,17191.0,0,0,0,0,1,6,> 50,64.0
2,400015912,3873005,561,201505,41.0,A_CON NEGOCIO/M,28.0,NaN,NaN,4662.0,7.0,NaN,OMEGA,OMEGA_M,201505,5897.0,0,0,0,0,1,6,<=50,41.0
3,400015973,776366,724,201505,44.0,C_CON NEGOCIO/F,NaN,1.0,18.0,0.0,1.0,0.104229,OMEGA,OMEGA_M,201505,8000.0,0,0,0,0,1,6,<=50,44.0
4,400014820,347497,713,201505,40.0,C_CON NEGOCIO/F,49.0,1.0,42.0,0.0,1.0,0.034422,OMEGA,OMEGA_M,201505,29686.0,0,0,0,0,1,6,<=40,40.0
5,400018477,1011052,631,201506,53.0,B_EMPLEADO O ASALARIADO/M,NaN,1.0,137.0,48701.0,9.0,0.000000,OMEGA,OMEGA_M,201506,13615.0,0,0,0,0,1,6,> 50,53.0
6,400018510,1800455250,661,201506,50.0,C_CON NEGOCIO/F,115.0,NaN,115.0,0.0,2.0,NaN,OMEGA,OMEGA_M,201506,31928.0,0,0,0,0,1,6,<=50,50.0
7,400021185,2964904,641,201507,63.0,C_CON NEGOCIO/F,293.0,1.0,443.0,0.0,1.0,0.148270,OMEGA,OMEGA_M,201507,10264.0,0,0,0,0,1,6,> 50,63.0
8,400021193,1800703853,682,201507,44.0,A_CON NEGOCIO/M,26.0,NaN,NaN,0.0,1.0,NaN,OMEGA,OMEGA_M,201507,29174.0,0,0,0,1,1,6,<=50,44.0
9,400021250,1124604,695,201507,46.0,D_EMPLEADO O ASALARIADO/F,90.0,NaN,90.0,0.0,1.0,NaN,OMEGA,OMEGA_M,201507,21731.0,0,0,0,0,1,6,<=50,46.0
